# InputScope Analysis

This notebook provides a means for analysing data generated using the [InputScope](https://github.com/suurjaak/InputScope) mouse and keyboard interaction logger. Data from the tool is stored in an SQLite database. This notebook provides the ability to combine outputs from one or multiple InputScope databases if required.

#### Import Packages

In [ ]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import os

#### Set paths to InputScope databases, screen images and analysis outputs

In [ ]:
db_filepath = "./InputScope/"
screen_content = "./ScreenContent/"
outputs = "./Outputs/"

#### Function to create the database connection

If you try to connect to an SQLite database file that does not exist, SQLite will automatically create the new database for you. However, any folder's specified in the filepath to the database must exist before you execute the program.

In [ ]:
# Define the function to connect to an SQLite database
def create_connection(db_file):
    
    # Create a new database connection object
    db = None
    try:
        # Try the database connection
        db = sqlite3.connect(db_file)
        
        # Query tables and store in dataframes
        app_events = pd.read_sql_query("SELECT * FROM app_events", db)
        counts = pd.read_sql_query("SELECT * FROM counts", db)
        screen_sizes = pd.read_sql_query("SELECT * FROM screen_sizes", db)
        clicks = pd.read_sql_query("SELECT * FROM clicks", db)
        moves = pd.read_sql_query("SELECT * FROM moves", db)
        scrolls = pd.read_sql_query("SELECT * FROM scrolls", db)
        keys = pd.read_sql_query("SELECT * FROM keys", db)
        combos = pd.read_sql_query("SELECT * FROM combos", db)
        
        # Create dictionary of datatables
        datatables = {"app_events": app_events,
                      "counts": counts,
                      "screen_sizes": screen_sizes,
                      "clicks": clicks,
                      "moves": moves,
                      "scrolls": scrolls,
                      "keys": keys,
                      "combos": combos}
        
        # Return the datatables
        return datatables
        
    # Catch and print any errors
    except Error as e:
        print(e)
    # Close the connection after execution of the try/except block
    finally:
        if db:
            db.close()

In [ ]:
db_filepath + "inputscope.db"#filename

#### Extract and merge contents from multiple InputScope databases

This code assumes that the separate SQLite databases have the structure.

In [ ]:
# Loop through database folders
for filename in os.listdir(db_filepath):
    db_tables = create_connection(db_filepath + filename)
    print("Read database - " + filename)

#### Display App Start Events

In [ ]:
db_tables["app_events"]

#### Display Counts of Interactions

In [ ]:
db_tables["counts"]

#### Display Screen Sizes

In [ ]:
db_tables["screen_sizes"]

#### Plot Heatmap

Visualisation based on the following: https://stackoverflow.com/questions/36957149/density-map-heatmaps-in-matplotlib/36958298

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy.stats.kde import gaussian_kde

x,y = db_tables["clicks"]["x"], db_tables["clicks"]["y"]

k = gaussian_kde(np.vstack([x,y]))
xi, yi = np.mgrid[x.min():x.max():x.size**0.5*1j,y.min():y.max():y.size**0.5*1j]
zi = k(np.vstack([xi.flatten(), yi.flatten()]))

# Custom colourmap can be used instead of Matplotlib defaults
colors = [(0, 0, 1), (0, 1, 1), (0, 1, 0.75), (0, 1, 0), (0.75, 1, 0),
          (1, 1, 0), (1, 0.8, 0), (1, 0.7, 0), (1, 0, 0)]
cm = LinearSegmentedColormap.from_list('sample', colors)

# Set figure size
fig = plt.figure(figsize=(14,16))
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

# cmap specifies the colourmap and alpha makes the plots semitransparent
ax1.pcolormesh(xi, yi, zi.reshape(xi.shape), cmap="viridis", alpha=0.5)
ax2.contourf(xi, yi, zi.reshape(xi.shape), cmap="viridis", alpha=0.5)

ax1.set_xlim(x.min(), x.max())
ax1.set_ylim(y.min(), y.max())
ax2.set_xlim(x.min(), x.max())
ax2.set_ylim(y.min(), y.max())

# Read image to overlay
im = plt.imread(screen_content + 'img.jpg')

# Show plot 
ax1.imshow(im, extent=[x.min(), x.max(), y.min(), y.max()], aspect='auto')
ax2.imshow(im, extent=[x.min(), x.max(), y.min(), y.max()], aspect='auto')

# Save plot
plt.savefig(outputs + 'clicks.jpg')